In [ ]:
import shapely.geometry

**API**

In [ ]:
import requests

with requests.session() as s:
  resp = s.get("https://opendata.vancouver.ca/api/explore/v2.1/catalog/datasets/business-improvement-areas-bia/records?limit=100")

  bia = resp.json()
print(bia['total_count'])
areas = bia['results']

areas

24


[{'mapid': '1004004',
  'name': 'Mount Pleasant BIA',
  'geom': {'type': 'Feature',
   'geometry': {'coordinates': [[[-123.10161430060336, 49.2664247711054],
      [-123.101633797894, 49.265972772055314],
      [-123.10165284115314, 49.265520728389724],
      [-123.10167177420303, 49.26506872975837],
      [-123.10169084423273, 49.26461666800134],
      [-123.10289196039258, 49.26463849742055],
      [-123.10292918325382, 49.26373396942495],
      [-123.10486390045767, 49.26376902008086],
      [-123.10488018470085, 49.26337191193224],
      [-123.10945096912525, 49.26345480003202],
      [-123.10945092609737, 49.263398058310806],
      [-123.11094931401813, 49.26342727584916],
      [-123.11096738332554, 49.26264667241648],
      [-123.1108505435678, 49.26266553058577],
      [-123.10946997573956, 49.2626398702429],
      [-123.10933316787373, 49.26261031559702],
      [-123.10545037167964, 49.262541533445905],
      [-123.10544709300844, 49.262640159863246],
      [-123.1052839203964

In [ ]:
from itertools import chain

from shapely import coords
for a in areas:
  coord = a['geom']['geometry']['coordinates']

  try:
    # print(len(coord))
    if len(coord) > 1:
      # print(coord[0])
      # print(coord[1])
      coord = coord[0]+coord[1]
    else:
      coord = coord[0]
    coord = [tuple(p) for p in coord]
    # print(len(res_coord), res_coord)
    polygon = shapely.geometry.Polygon(coord)
    a['shapely_polygon'] = polygon
  except Exception as e:
    arr1 = [tuple(p) for p in a['geom']['geometry']['coordinates'][0][0]]
    arr2 = [tuple(p) for p in a['geom']['geometry']['coordinates'][1][0]]
    coord = arr1 + arr2
    polygon = shapely.geometry.Polygon(coord)
    a['shapely_polygon'] = polygon

In [ ]:
import requests

with requests.session() as s:
  resp = s.get("https://opendata.vancouver.ca/api/explore/v2.1/catalog/datasets/disability-parking/records?limit=100")
  dp = resp.json()

with requests.session() as s:
  resp = s.get("https://opendata.vancouver.ca/api/explore/v2.1/catalog/datasets/disability-parking/records?limit=100&offset=100")

  dp2 = resp.json()
len(dp2)


print(len(dp['results']), len(dp2['results']))
parkings = dp['results'] + dp2['results']
len(parkings)

100 28


128

In [ ]:
#parkings
for park in parkings:
  park['shapely_point'] = shapely.geometry.Point(*park['geom']['geometry']['coordinates'])

parkings[0]

{'description': 'Designated meter parking space',
 'geom': {'type': 'Feature',
  'geometry': {'coordinates': [-123.069865, 49.27218], 'type': 'Point'},
  'properties': {}},
 'location': 'North Side 1600 Kitchener St',
 'notes': 'No stopping accessible zone',
 'spaces': 1,
 'url': None,
 'geo_local_area': 'Grandview-Woodland',
 'geo_point_2d': {'lon': -123.069865, 'lat': 49.27218},
 'shapely_point': <POINT (-123.07 49.272)>}

In [ ]:
for park in parkings:
  park['bia_name'] = park['bia_id'] = None
  for area in areas:
    polygon = area.get('shapely_polygon')
    if polygon:
      if park['shapely_point'].within(polygon):
        park['bia_name'] = area['name']
        park['bia_id'] = area['mapid']
        break

In [ ]:
i = 1

for park in parkings:
  if park.get('bia_name'):
    print(i, park['location'], "---------", park['bia_name'])
    i+=1

1 North Side 1600 Kitchener St --------- Commercial Dr. BIA
2 South Side 1700 Parker St --------- Commercial Dr. BIA
3 West Side  2500 Main St --------- Mount Pleasant BIA
4 South Side 2400 W 41st Av --------- Kerrisdale BIA
5 South Side 200 E Pender St --------- Chinatown BIA
6 South Side 1900 W 4th Av --------- Kitsilano Fourth Ave. BIA
7 West Side  700 Broughton St --------- West End BIA
8 West Side  400 Slocan St --------- Hastings - North BIA
9 South Side 2600 W Broadway --------- West Broadway BIA
10 East Side  500 Richards St --------- Downtown Vancouver BIA
11 North Side 1400 W 15th Av --------- South Granville BIA
12 East Side  4200 Dunbar St --------- Dunbar Village BIA
13 East Side  1100 Burrard St --------- Downtown Vancouver BIA
14 West Side  5600 Yew St --------- Kerrisdale BIA
15 North Side 3200 W Broadway --------- West Broadway BIA
16 South Side 100 E 13th Av --------- Mount Pleasant BIA
17 East Side  1100 Homer St --------- Yaletown BIA
18 South Side 200 E 10th Av ---

In [ ]:
final_data = [{'location': parking['location'], 'bia_name': parking['bia_name'], 'bia_id': parking['bia_id']} for parking in parkings]

In [ ]:
final_data

[{'location': 'North Side 1600 Kitchener St',
  'bia_name': 'Commercial Dr. BIA',
  'bia_id': '1004005'},
 {'location': 'South Side 1700 Parker St',
  'bia_name': 'Commercial Dr. BIA',
  'bia_id': '1004005'},
 {'location': 'East Side  5400 Vine St', 'bia_name': None, 'bia_id': None},
 {'location': 'West Side  5300 Yew St', 'bia_name': None, 'bia_id': None},
 {'location': 'West Side  2500 Main St',
  'bia_name': 'Mount Pleasant BIA',
  'bia_id': '1004004'},
 {'location': 'South Side 2400 W 41st Av',
  'bia_name': 'Kerrisdale BIA',
  'bia_id': '1004011'},
 {'location': 'East Side  4200 Main St', 'bia_name': None, 'bia_id': None},
 {'location': 'South Side 200 E Pender St',
  'bia_name': 'Chinatown BIA',
  'bia_id': '1004021'},
 {'location': 'South Side 1900 W 4th Av',
  'bia_name': 'Kitsilano Fourth Ave. BIA',
  'bia_id': '1004013'},
 {'location': 'West Side  700 Broughton St',
  'bia_name': 'West End BIA',
  'bia_id': '1004017'},
 {'location': 'West Side  400 Slocan St',
  'bia_name': '

In [ ]:
import pandas as pd

df = pd.DataFrame(final_data)

In [ ]:
df.to_csv("disability-parking-mapped-bia.csv", index=False)